# Day 3: EDA & Baseline Models
## Nairobi House Prediction Project

**Date:** [To be filled]  
**Author:** [Your Name]

## Objectives
1. Perform exploratory data analysis
2. Visualize key relationships
3. Build baseline models
4. Evaluate model performance

## 1. Import Libraries

In [ ]:
# Data manipulation

# Machine learning

# Visualization

## 2. Load Processed Data

In [ ]:
# Load data

## 3. Exploratory Data Analysis

### 3.1 Target Variable Distribution

In [ ]:
# Analyze target variable

### 3.2 Feature Distributions

In [ ]:
# Visualize feature distributions

### 3.3 Correlation Analysis

In [ ]:
# Correlation heatmap

### 3.4 Relationship with Target

In [ ]:
# Scatter plots with target

## 4. Prepare Data for Modeling

In [ ]:
# Train-test split

## 5. Baseline Models

### 5.1 Linear Regression

In [ ]:
# Train linear regression

### 5.2 Decision Tree

In [ ]:
# Train decision tree

### 5.3 Random Forest

In [ ]:
# Train random forest

## 6. Model Evaluation

In [ ]:
# Compare models

## 7. Save Models

In [ ]:
# Save best baseline model

## Summary

### Key Insights from EDA
- [To be filled]

### Baseline Model Performance
- [To be filled]

### Next Steps
- [To be filled]